In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT
import pandas as pd
import numpy as np
import os
import re


In [2]:
qasm_path = r"./qasm_output"
qpy_path = r"./qpy_output"

In [46]:
test = "canonical_ae_mapped_ibm_opt2_3.qasm"

In [71]:
get_opt_level(test)

canonical_ae_mapped_ibm_opt2_3.qasm None
canonical_ae_mapped_ibm_opt2_3.qasm


UnboundLocalError: local variable 'num' referenced before assignment

In [70]:
def get_opt_level(filename:str):
    pat = re.compile(r'[0-9]')
    m = pat.search(filename)
    print (filename, m)
    if m:
        num = m.group()[1:-1]
    else:
        print (filename)
    print(num)
    return num

In [24]:
def get_num_qubits(filename:str):
    pat = re.compile(r'_\d+.')
    m = pat.search(filename)
    if m:
        num = m.group()[1:-1]
    else:
        print (filename)
    return num

In [25]:
def parse_data(directory: str, filename:str):
    benchmark = filename.split("_")[0]
    num_qubits = get_num_qubits(filename)
    algorithm_flag =  "algorithm" in filename
    transpiled_flag = "transpiled" in filename 
    mapped_flag = "mapped" in filename
    opt_level = get_opt_level(filename)
    path = os.path.join(directory, filename)
    if ("ibm" in filename): gate_set="ibm"
    elif ("rigetti" in filename): gate_set="ibm"
    parsed_data = [benchmark, num_qubits, gate_set, algorithm_flag, transpiled_flag, mapped_flag, opt_level, path]
    return parsed_data

In [26]:
directory = qasm_path
rows_list = []
for filename in os.listdir(directory):
    if filename.endswith(".qasm") or filename.endswith(".qpy"):
        #print(filename)
        parsed_data = parse_data(directory, filename)
        rows_list.append(parsed_data)
        continue
    else:
        #print(filename)
        continue
print(rows_list)

canonical_ae_mapped_ibm_opt2_3.qasm None
canonical_ae_mapped_ibm_opt2_3.qasm


UnboundLocalError: local variable 'num' referenced before assignment

In [ ]:
colnames = ["Benchmark", "Num Qubits", "Algo_layer", "Transpiled_layer", "Mapped_layer", "Path"]

In [ ]:
print(pd.DataFrame(rows_list, columns = colnames))